In [6]:
import pandas as pd
import numpy as np
from pymongo import MongoClient
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

In [7]:
mongo_client = MongoClient("mongodb://localhost:27017/")
mongo_db = mongo_client["traffic"]
features_coll = mongo_db["traffic_features"]

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Sử dụng thiết bị: {device}")

Sử dụng thiết bị: cuda


In [9]:
required_columns = [
    'count_point_id', 'year', 'month', 'hour', 'day_of_week', 'is_weekend',
    'road_type', 'region_name', 'local_authority_name', 'link_length_km',
    'pedal_cycles', 'two_wheeled_motor_vehicles', 'cars_and_taxis',
    'buses_and_coaches', 'lgvs', 'hgvs_2_rigid_axle', 'all_hgvs',
    'all_motor_vehicles' # Target
]


In [10]:
# Tạo projection dictionary: {tên_trường: 1} để bao gồm
projection = {col: 1 for col in required_columns}
projection['_id'] = 0  # Thường thì không cần _id, loại bỏ nó sẽ giảm tải

print("Đang tải dữ liệu từ MongoDB với projection...")
# Thêm projection vào câu lệnh find()
cursor = features_coll.find({}, projection)
df = pd.DataFrame(list(cursor))
print(f"Tải thành công {len(df)} bản ghi.")

Đang tải dữ liệu từ MongoDB với projection...
Tải thành công 4253812 bản ghi.


In [ ]:
# Định nghĩa các cột features và target
features = [
    "hour", "month", "is_weekend",
    "pedal_cycles", "two_wheeled_motor_vehicles",
    "cars_and_taxis", "buses_and_coaches", "lgvs",
    "hgvs_2_rigid_axle", "all_hgvs"
]
categorical_cols = ["day_of_week", "road_type", "region_name", "local_authority_name"]
target = "all_motor_vehicles"

In [ ]:
# --- BƯỚC LÀM SẠCH QUAN TRỌNG NHẤT ---
# Chuyển đổi các cột số và xử lý giá trị "bẩn"
numeric_cols_to_clean = features + [target, 'count_point_id', 'year', 'link_length_km']
for col in numeric_cols_to_clean:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

In [ ]:
# Kiểm tra và xử lý NaN
print("Số lượng giá trị NaN trước khi xử lý:")
print(df.isnull().sum())
df.fillna(0, inplace=True)
print("\nSố lượng giá trị NaN sau khi xử lý: Hoàn tất.")

# Xử lý các cột phân loại
for col in categorical_cols:
    if col in df.columns:
        df[col] = LabelEncoder().fit_transform(df[col].astype(str))
        features.append(col) # Thêm cột đã mã hóa vào danh sách features

print("Danh sách features cuối cùng để huấn luyện:")
print(features)

Số lượng giá trị NaN trước khi xử lý:
count_point_id                      0
year                                0
hour                                0
region_name                         0
local_authority_name                0
road_type                           0
link_length_km                2317688
pedal_cycles                        0
two_wheeled_motor_vehicles          0
cars_and_taxis                      1
buses_and_coaches                   2
lgvs                                0
hgvs_2_rigid_axle                   4
all_hgvs                            9
all_motor_vehicles                 12
day_of_week                         0
is_weekend                          0
month                               0
dtype: int64

Số lượng giá trị NaN sau khi xử lý: Hoàn tất.
Danh sách features cuối cùng để huấn luyện:
['hour', 'month', 'is_weekend', 'pedal_cycles', 'two_wheeled_motor_vehicles', 'cars_and_taxis', 'buses_and_coaches', 'lgvs', 'hgvs_2_rigid_axle', 'all_hgvs', 'day_of_week', '

In [ ]:
# 3️⃣ GIAI ĐOẠN 1: HUẤN LUYỆN LSTM (PYTORCH) ĐỂ TẠO FEATURE
# =========================================================
print("\nBắt đầu giai đoạn 1: Huấn luyện LSTM trên từng chuỗi thời gian...")

# --- Định nghĩa Class mô hình LSTM ---
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out



Bắt đầu giai đoạn 1: Huấn luyện LSTM trên từng chuỗi thời gian...


In [ ]:
# --- Bắt đầu vòng lặp huấn luyện ---
df = df.sort_values(by=['count_point_id', 'year', 'month', 'hour']).reset_index(drop=True)
unique_point_ids = df['count_point_id'].unique()
all_lstm_results = []
n_steps = 5

In [ ]:
for point_id in tqdm(unique_point_ids, desc="Huấn luyện LSTM (PyTorch)"):
    df_point = df[df['count_point_id'] == point_id].copy()
    
    if len(df_point) < n_steps + 15: # Cần đủ dữ liệu để tạo chuỗi và huấn luyện
        continue

    # =========================================================
    # BƯỚC 1: CHUẨN BỊ DỮ LIỆU CHO MÔ HÌNH
    # =========================================================
    data_to_scale = df_point[features + [target]]
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(data_to_scale)

    # Tạo chuỗi X, y
    X_lstm, y_lstm = [], []
    for i in range(len(scaled_data) - n_steps):
        X_lstm.append(scaled_data[i:i+n_steps, :-1])
        y_lstm.append(scaled_data[i+n_steps, -1])
    
    if len(X_lstm) == 0: 
        continue
    
    # Chuyển dữ liệu sang PyTorch Tensors
    X_tensor = torch.FloatTensor(np.array(X_lstm)).to(device)
    y_tensor = torch.FloatTensor(np.array(y_lstm)).unsqueeze(1).to(device)

    # Tạo DataLoader
    dataset = TensorDataset(X_tensor, y_tensor)
    dataloader = DataLoader(dataset, batch_size=64, shuffle=True)
    
    # =========================================================
    # BƯỚC 2: KHỞI TẠO VÀ HUẤN LUYỆN MÔ HÌNH
    # =========================================================
    model_lstm = LSTMModel(input_size=X_tensor.shape[2], hidden_size=32, num_layers=1, output_size=1).to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model_lstm.parameters(), lr=0.001)
    
    num_epochs = 5
    for epoch in range(num_epochs):
        for inputs, labels in dataloader:
            outputs = model_lstm(inputs)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    # =========================================================
    # BƯỚC 3: DỰ BÁO VÀ XỬ LÝ KẾT QUẢ
    # =========================================================
    model_lstm.eval() # Chuyển sang chế độ đánh giá
    with torch.no_grad():
        # BIẾN `predicted_scaled` ĐƯỢC TẠO RA Ở ĐÂY!
        predicted_scaled = model_lstm(X_tensor).cpu().numpy()
    
    # Khôi phục giá trị gốc (Bây giờ code này sẽ hoạt động)
    inverse_helper = np.zeros((len(predicted_scaled), len(features) + 1))
    inverse_helper[:, -1] = predicted_scaled.flatten()
    predicted_volume_lstm = scaler.inverse_transform(inverse_helper)[:, -1]

    # Lưu kết quả
    df_point_result = df_point.iloc[n_steps:].copy()
    df_point_result['lstm_predicted_volume'] = predicted_volume_lstm
    all_lstm_results.append(df_point_result)


# --- KẾT THÚC VÒNG LẶP ---
# Ghép tất cả kết quả từ LSTM lại
df_stacked = pd.concat(all_lstm_results, ignore_index=True)
print(f"\nHoàn tất giai đoạn 1. Đã tạo được {len(df_stacked)} bản ghi với feature từ LSTM.")

Huấn luyện LSTM (PyTorch): 100%|██████████| 41647/41647 [20:10<00:00, 34.40it/s]



Hoàn tất giai đoạn 1. Đã tạo được 4043687 bản ghi với feature từ LSTM.


In [ ]:
# 4️⃣ GIAI ĐOẠN 2: HUẤN LUYỆN XGBOOST VỚI FEATURE TỪ LSTM
# =========================================================
from sklearn.model_selection import train_test_split

print("\nBắt đầu giai đoạn 2: Huấn luyện XGBoost với feature được làm giàu...")

# --- Chuẩn bị dữ liệu cho XGBoost ---
# Feature mới của chúng ta là 'lstm_predicted_volume'
features_for_xgb = features + ['lstm_predicted_volume']
target_for_xgb = target

# Đảm bảo không có giá trị NaN/inf nào còn sót lại
df_stacked.replace([np.inf, -np.inf], np.nan, inplace=True)
df_stacked.fillna(0, inplace=True)

# Lấy dữ liệu X và y cuối cùng
X = df_stacked[features_for_xgb]
y = df_stacked[target_for_xgb]

# --- Chia dữ liệu thành tập huấn luyện và tập kiểm tra ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Kích thước tập huấn luyện XGBoost: {X_train.shape}")
print(f"Kích thước tập kiểm tra XGBoost: {X_test.shape}")


Bắt đầu giai đoạn 2: Huấn luyện XGBoost với feature được làm giàu...
Kích thước tập huấn luyện XGBoost: (3234949, 15)
Kích thước tập kiểm tra XGBoost: (808738, 15)


In [ ]:
# --- Khởi tạo và Huấn luyện mô hình XGBoost ---
model_xgb_stacked = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=1000,         # Tăng số lượng cây để mô hình học sâu hơn
    max_depth=7,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1,                 # Sử dụng tất cả các CPU cores
    # Thêm tham số tree_method để sử dụng GPU nếu có
    tree_method='gpu_hist' if device.type == 'cuda' else 'auto'
)


In [ ]:
print("\nĐang huấn luyện XGBoost...")
# Huấn luyện mô hình với early stopping
model_xgb_stacked.fit(X_train, y_train,
                    eval_set=[(X_test, y_test)], # Dừng nếu sau 50 vòng mà hiệu suất không cải thiện
                    verbose=100) # In ra kết quả sau mỗi 100 vòng



Đang huấn luyện XGBoost...


c:\Users\DELL\miniconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:07:05] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  bst.update(dtrain, iteration=i, fobj=obj)


[0]	validation_0-rmse:795.61793
[100]	validation_0-rmse:38.90523
[200]	validation_0-rmse:36.35525
[300]	validation_0-rmse:35.46785
[400]	validation_0-rmse:35.02281
[500]	validation_0-rmse:34.80072
[600]	validation_0-rmse:34.66774
[700]	validation_0-rmse:34.59657
[800]	validation_0-rmse:34.55204
[900]	validation_0-rmse:34.56449
[999]	validation_0-rmse:34.55129


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.05, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=7,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=1000,
             n_jobs=-1, num_parallel_tree=None, ...)

In [ ]:
# 5️⃣ ĐÁNH GIÁ MÔ HÌNH KẾT HỢP VÀ DỰ ĐOÁN
# =========================================================
print("\nĐánh giá hiệu suất mô hình kết hợp và thực hiện dự đoán...")

# Dự báo trên tập kiểm tra
y_pred_stacked = model_xgb_stacked.predict(X_test)

# Tính toán các chỉ số hiệu suất
mse = mean_squared_error(y_test, y_pred_stacked)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred_stacked)

print("\n--- KẾT QUẢ ĐÁNH GIÁ MÔ HÌNH CUỐI CÙNG ---")
print(f"  - Mean Squared Error (MSE): {mse:.2f}")
print(f"  - Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"  - R-squared (R²): {r2:.4f}")
print("---------------------------------------------")


Đánh giá hiệu suất mô hình kết hợp và thực hiện dự đoán...


c:\Users\DELL\miniconda3\Lib\site-packages\xgboost\core.py:2676: UserWarning: [15:12:41] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  if len(data.shape) != 1 and self.num_features() != data.shape[1]:
c:\Users\DELL\miniconda3\Lib\site-packages\xgboost\core.py:729: UserWarning: [15:12:41] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  return func(**kwargs)



--- KẾT QUẢ ĐÁNH GIÁ MÔ HÌNH CUỐI CÙNG ---
  - Mean Squared Error (MSE): 1193.79
  - Root Mean Squared Error (RMSE): 34.55
  - R-squared (R²): 0.9983
---------------------------------------------


In [ ]:
# 6️⃣ PHÂN TÍCH KẾT QUẢ DỰ ĐOÁN VÀ PHÂN LOẠI ÙN TẮC
# =========================================================
print("\nPhân tích kết quả dự đoán trên tập kiểm tra...")

# Tạo một DataFrame kết quả để dễ phân tích
df_final_result = X_test.copy()
df_final_result['actual_volume'] = y_test
df_final_result['predicted_volume'] = y_pred_stacked
df_final_result['predicted_volume'] = df_final_result['predicted_volume'].apply(lambda x: max(0, x)) # Đảm bảo không âm
df_final_result['error'] = df_final_result['actual_volume'] - df_final_result['predicted_volume']

# Thêm lại thông tin 'lstm_predicted_volume' để so sánh
df_final_result['lstm_predicted_volume'] = df_stacked.loc[df_final_result.index, 'lstm_predicted_volume']


Phân tích kết quả dự đoán trên tập kiểm tra...


In [ ]:
# Định nghĩa quy tắc phân loại ùn tắc
# Bạn nên điều chỉnh các ngưỡng này dựa trên df['all_motor_vehicles'].describe()
def classify_congestion(volume):
    if volume < 50:
        return "1 - Rất thông thoáng"
    elif volume < 150:
        return "2 - Thông thoáng"
    elif volume < 300:
        return "3 - Đông đúc"
    else:
        return "4 - Ùn tắc"

df_final_result['congestion_level'] = df_final_result['predicted_volume'].apply(classify_congestion)

print("\n5 dòng kết quả dự báo cuối cùng:")
print(df_final_result[[
    'actual_volume', 
    'lstm_predicted_volume', # Dự báo của LSTM
    'predicted_volume',      # Dự báo cuối cùng của XGBoost
    'error',                 # Sai số
    'congestion_level'
]].head())


5 dòng kết quả dự báo cuối cùng:
         actual_volume  lstm_predicted_volume  predicted_volume     error  \
1118089          704.0             779.353698        700.983582  3.016418   
3318591           56.0              47.042337         55.524445  0.475555   
2366743           92.0              81.109572         93.991364 -1.991364   
572103           167.0             161.211343        166.162811  0.837189   
3160545           15.0              24.794047         15.284058 -0.284058   

             congestion_level  
1118089            4 - Ùn tắc  
3318591      2 - Thông thoáng  
2366743      2 - Thông thoáng  
572103           3 - Đông đúc  
3160545  1 - Rất thông thoáng  


In [ ]:
import os
os.makedirs("saved_models/lstm", exist_ok=True)

for point_id in unique_point_ids:
    model_path = f"saved_models/lstm/model_lstm_{point_id}.pth"
    torch.save(model_lstm.state_dict(), model_path)


In [ ]:
torch.save(model_lstm.state_dict(), "model_lstm.pth")
print("✅ Đã lưu mô hình LSTM thành công.")


✅ Đã lưu mô hình LSTM thành công.


In [ ]:
import joblib
joblib.dump(scaler, "scaler.pkl")
print("✅ Đã lưu scaler thành công.")


✅ Đã lưu scaler thành công.


In [ ]:
import joblib
joblib.dump(model_xgb_stacked, "model_xgb_stacked.pkl")
print("✅ Đã lưu mô hình XGBoost (pickle) thành công.")


✅ Đã lưu mô hình XGBoost (pickle) thành công.


In [ ]:
# Tạo thư mục nếu chưa có
os.makedirs("model", exist_ok=True)

# Lưu ra file JSON
model_xgb_stacked.save_model("model/model_xgb_stacked.json")
print("✅ Model XGBoost đã được lưu thành công dưới dạng JSON.")


c:\Users\DELL\miniconda3\Lib\site-packages\xgboost\sklearn.py:1028: UserWarning: [17:07:23] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  self.get_booster().save_model(fname)


✅ Model XGBoost đã được lưu thành công dưới dạng JSON.


In [ ]:
# Khi train
input_size = X_tensor.shape[2]   # Nếu X_train có dạng (samples, time_steps, features)
model_lstm_loaded = LSTMModel(input_size, hidden_size=32, num_layers=1, output_size=1)

In [ ]:

model_lstm_loaded.load_state_dict(torch.load("model_lstm.pth", map_location=device))
model_lstm_loaded.to(device)
model_lstm_loaded.eval()
print("✅ Đã tải lại mô hình LSTM thành công.")


✅ Đã tải lại mô hình LSTM thành công.


C:\Users\DELL\AppData\Local\Temp\ipykernel_24428\876044939.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_lstm_loaded.load_state_dict(torch.load("model_lstm.pth",

In [ ]:
import torch
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 🔹 Lấy 5 giá trị volume cuối cùng trong dữ liệu thực tế
last_sequence = df["all_motor_vehicles"].values[-n_steps:]

scaler_volume = MinMaxScaler()
scaler_volume.fit(df[["all_motor_vehicles"]])

last_sequence = df["all_motor_vehicles"].values[-n_steps:]
scaled_last_seq = scaler_volume.transform(df[["all_motor_vehicles"]].iloc[-n_steps:])


In [ ]:
print(df.columns.tolist())


['count_point_id', 'year', 'hour', 'region_name', 'local_authority_name', 'road_type', 'link_length_km', 'pedal_cycles', 'two_wheeled_motor_vehicles', 'cars_and_taxis', 'buses_and_coaches', 'lgvs', 'hgvs_2_rigid_axle', 'all_hgvs', 'all_motor_vehicles', 'day_of_week', 'is_weekend', 'month']


In [ ]:
# 7️⃣ XÂY DỰNG CHỨC NĂNG DỰ BÁO THỰC TẾ
# =========================================================
print("\n" + "="*50)
print("XÂY DỰNG CHỨC NĂNG DỰ BÁO THỰC TẾ")
print("="*50)

# --- A. Lưu trữ các đối tượng cần thiết cho việc dự báo ---
# 1. Lưu lại các LabelEncoder đã fit
label_encoders = {}
for col in categorical_cols:
    if col in df.columns:
        le = LabelEncoder()
        le.fit(df[col].astype(str)) # Fit trên toàn bộ dữ liệu gốc
        label_encoders[col] = le
        
print("Đã lưu các LabelEncoder.")


XÂY DỰNG CHỨC NĂNG DỰ BÁO THỰC TẾ
Đã lưu các LabelEncoder.


In [ ]:
def predict_next_day(count_point_id, df, features, model_lstm, model_xgb_stacked, scaler, device, n_steps=5):
    """
    Dự báo lưu lượng xe cho 24 giờ tiếp theo tại một điểm đếm cụ thể (count_point_id).

    Args:
        count_point_id (int): ID của điểm đếm
        df (pd.DataFrame): dữ liệu gốc đã huấn luyện
        features (list): danh sách cột features
        model_lstm (torch.nn.Module): mô hình LSTM đã huấn luyện
        model_xgb_stacked: mô hình XGBoost đã huấn luyện
        scaler: MinMaxScaler đã fit trên dữ liệu
        device: torch.device
        n_steps (int): số bước chuỗi LSTM

    Returns:
        pd.DataFrame: dự báo 24 giờ tiếp theo
    """
    df_point = df[df['count_point_id'] == count_point_id].sort_values(by=['year','month','hour']).reset_index(drop=True)
    
    if len(df_point) < n_steps:
        raise ValueError(f"Không đủ dữ liệu cho count_point_id={count_point_id} để tạo chuỗi LSTM.")

    results = []
    
    for hour in range(24):
        # Lấy n_steps cuối để LSTM dự báo
        last_seq = df_point[features].iloc[-n_steps:].replace([np.inf, -np.inf], 0).fillna(0).astype(float).values
        last_seq_tensor = torch.FloatTensor(last_seq).unsqueeze(0).to(device)
        
        # Dự báo LSTM
        model_lstm.eval()
        with torch.no_grad():
            lstm_pred_scaled = model_lstm(last_seq_tensor).cpu().numpy().flatten()
        
        # Khôi phục giá trị gốc
        inverse_helper = np.zeros((len(lstm_pred_scaled), len(features)+1))
        inverse_helper[:, -1] = lstm_pred_scaled
        lstm_pred_volume = scaler.inverse_transform(inverse_helper)[:, -1][0]
        
        # Chuẩn bị row cho XGBoost
        last_row = df_point[features].iloc[[-1]].copy()
        last_row['lstm_predicted_volume'] = lstm_pred_volume
        last_row = last_row.replace([np.inf, -np.inf], 0).fillna(0).astype(float)
        X_xgb = last_row[features + ['lstm_predicted_volume']].values.reshape(1, -1)
        
        # Dự báo bằng XGBoost
        xgb_pred = model_xgb_stacked.predict(X_xgb)[0]
        xgb_pred = max(0, xgb_pred)  # đảm bảo >= 0

        # Phân loại mức độ ùn tắc
        if xgb_pred < 50:
            congestion = "1 - Rất thông thoáng"
        elif xgb_pred < 150:
            congestion = "2 - Thông thoáng"
        elif xgb_pred < 300:
            congestion = "3 - Đông đúc"
        else:
            congestion = "4 - Ùn tắc"
        
        # Lưu kết quả
        results.append({
            "hour": hour,
            "lstm_predicted_volume": lstm_pred_volume,
            "predicted_volume": xgb_pred,
            "congestion_level": congestion
        })
        
        # Cập nhật df_point với giá trị dự báo để dùng cho giờ tiếp theo
        next_row = last_row.copy()
        next_row['hour'] = hour
        next_row['all_motor_vehicles'] = xgb_pred
        df_point = pd.concat([df_point, next_row], ignore_index=True)
    
    return pd.DataFrame(results)


In [ ]:
# Khởi tạo XGBRegressor rỗng
model_xgb_stacked_loaded = xgb.XGBRegressor()

# Load model từ JSON
model_xgb_stacked_loaded.load_model("model/model_xgb_stacked.json")

# Nếu muốn dùng GPU khi predict
model_xgb_stacked_loaded.tree_method = 'gpu_hist'

print("✅ Model XGBoost đã được load thành công từ JSON.")


✅ Model XGBoost đã được load thành công từ JSON.


In [ ]:
# 🔹 Dự báo 24 giờ tiếp theo
result_df = predict_next_day(
    count_point_id=54,
    df=df,
    features=features,
    model_lstm=model_lstm,                  # LSTM đã load hoặc train sẵn
    model_xgb_stacked=model_xgb_stacked_loaded,  # Dùng model đã load
    scaler=scaler,
    device=device,
    n_steps=10
)

C:\Users\DELL\AppData\Local\Temp\ipykernel_24428\2908272603.py:27: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  last_seq = df_point[features].iloc[-n_steps:].replace([np.inf, -np.inf], 0).fillna(0).astype(float).values
C:\Users\DELL\AppData\Local\Temp\ipykernel_24428\2908272603.py:43: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  last_row = last_row.replace([np.inf, -np.inf], 0).fillna(0).astype(float)
C:\Users\DELL\AppData\Local\Temp\ipykernel_24428\2908272603.py:27: FutureWarning: Downcasting behavior in `replace` is deprecated

In [ ]:
print(result_df)

    hour  lstm_predicted_volume  predicted_volume      congestion_level
0      0              72.024397         17.236866  1 - Rất thông thoáng
1      1               5.527166         15.644845  1 - Rất thông thoáng
2      2               5.119615         15.644845  1 - Rất thông thoáng
3      3               2.792055         15.644845  1 - Rất thông thoáng
4      4              10.234891         15.644845  1 - Rất thông thoáng
5      5              15.908782         15.644845  1 - Rất thông thoáng
6      6              16.679357         15.644845  1 - Rất thông thoáng
7      7              23.087625         15.422552  1 - Rất thông thoáng
8      8              32.009406         15.702405  1 - Rất thông thoáng
9      9              42.645827         15.896573  1 - Rất thông thoáng
10    10              54.042291         16.096846  1 - Rất thông thoáng
11    11              54.828234         16.096846  1 - Rất thông thoáng
12    12              55.217444         16.096846  1 - Rất thông

In [11]:
print(df.columns.tolist())

['count_point_id', 'year', 'hour', 'region_name', 'local_authority_name', 'road_type', 'link_length_km', 'pedal_cycles', 'two_wheeled_motor_vehicles', 'cars_and_taxis', 'buses_and_coaches', 'lgvs', 'hgvs_2_rigid_axle', 'all_hgvs', 'all_motor_vehicles', 'day_of_week', 'is_weekend', 'month']
